# 서울시 생활인구
## 서울 생활인구 현황 (2022.09.28. 기준)
### url: https://data.seoul.go.kr/dataVisual/seoul/seoulLivingPopulation.do
### 서울시 생활인구란? 
    - 서울시와 KT가 공공빅데이터와 통신데이터를 이용하여 추계한 서울의 특정지역, 특정시점에 존재하는 모든 인구

1. 집계구 단위 서울 생활인구(내국인)
    - url: https://data.seoul.go.kr/dataList/OA-14979/F/1/datasetView.do
    - 설명: 서울시가 보유한 공공데이터와 통신데이터로 측정한 특정시점에 서울의 특정 지역에 존재하는 인구 중 내국인

<!-- <br> -->


2. 집계구 단위 서울 생활인구(장기체류 외국인)
    - url: https://data.seoul.go.kr/dataList/OA-14978/F/1/datasetView.do
    - 설명:서울시가 보유한 공공데이터와 통신데이터로 측정한 특정시점에 서울의 특정 지역에 존재하는 인구 중 장기체류 외국인

<!-- <br> -->


3. 집계구 단위 서울 생활인구(단기체류 외국인)
    - url: https://data.seoul.go.kr/dataList/OA-14980/F/1/datasetView.do
    - 설명:서울시가 보유한 공공데이터와 통신데이터로 측정한 특정시점에 서울의 특정 지역에 존재하는 인구 중 단기체류 외국인

<!-- <br> -->



※ 개인정보 비 식별화를 위하여 ‘3명’ 이하인 경우 “ * ” 처리


In [1]:
import os
import glob
import pandas as pd
import numpy as np

In [2]:
from tqdm.auto import tqdm, trange
from time import sleep

## 1. 집계구 단위 서울 생활인구(내국인)
    - a)2022년 01월~08월
    - b)2019년, 2020년, 2021년
    - 2019년 10월 15-27일 데이터 없음
## 2. 집계구 단위 서울 생활인구(장기체류 외국인)
    - a)2022년 01월~08월
    - b)2019년, 2020년, 2021년
## 3. 집계구 단위 서울 생활인구(단기체류 외국인)
    - a)2022년 01월~08월
    - b)2019년, 2020년, 2021년

In [3]:
# 폴더 안 파일명 리스트 가져오기
local_people_2019_list = []
local_people_2020_list = []
local_people_2021_list = []

long_foreigner_list = []
temp_foreinger_list= []
pbar = tqdm(os.listdir(r"./data/집계구 단위 서울 생활인구/local_people/2019"))
for file in pbar:
    if file.endswith(".csv"):
        pbar.set_description(file)
        local_people_2019_list.append(file)
        
pbar = tqdm(os.listdir(r"./data/집계구 단위 서울 생활인구/local_people/2020"))
for file in pbar:
    if file.endswith(".csv"):
        pbar.set_description(file)
        local_people_2020_list.append(file)
        
pbar = tqdm(os.listdir(r"./data/집계구 단위 서울 생활인구/local_people/2021"))
for file in pbar:
    if file.endswith(".csv"):
        pbar.set_description(file)
        local_people_2021_list.append(file)

pbar = tqdm(os.listdir(r"./data/집계구 단위 서울 생활인구/long_foreigner/"))
for file in pbar:
    if file.endswith(".csv"):
        pbar.set_description(file)
        long_foreigner_list.append(file)
 
pbar = tqdm(os.listdir(r"./data/집계구 단위 서울 생활인구/temp_foreigner/"))
for file in pbar:
    if file.endswith(".csv"):
        pbar.set_description(file)
        temp_foreinger_list.append(file)

local_people_2019_list.sort()
local_people_2020_list.sort()
local_people_2021_list.sort()
long_foreigner_list.sort()
temp_foreinger_list.sort()

print("no. of local_people_2019 >>", len(local_people_2019_list))
print("no. of local_people_2020 >>", len(local_people_2020_list))
print("no. of local_people_2021 >>", len(local_people_2021_list))

print("no. of long_foreigner_list >>", len(long_foreigner_list))
print("no. of temp_foreinger_list >>", len(temp_foreinger_list))

  0%|          | 0/352 [00:00<?, ?it/s]

  0%|          | 0/366 [00:00<?, ?it/s]

  0%|          | 0/365 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

no. of local_people_2019 >> 351
no. of local_people_2020 >> 366
no. of local_people_2021 >> 365
no. of long_foreigner_list >> 1
no. of temp_foreinger_list >> 1


In [4]:
def data_process(file_list, path):
    df_sum_per_day_집계구_list = []
    df_sum_per_day_행정동_list = []
    problem_list = []
    pbar = tqdm(file_list)
    for file in pbar:
        pbar.set_description(file) # pbar description에 작업중인 파일명 업데이트
        try:df_temp = pd.read_csv(path + file, encoding="utf-8", na_values="*")
        except UnicodeDecodeError:df_temp = pd.read_csv(path + file, encoding="euc-kr", na_values="*")
        except Exception as e: 
            # df_temp = pd.read_csv(path + file, encoding="euc-kr", na_values="*")
            print(file, ">>", e)
            problem_list.append(file)
            continue
        df_temp.rename(columns={'?"기준일ID"' : '기준일ID'})
        #결측값 0으로 대체
        df_temp.fillna(0, inplace=True)
        try:
            df_temp = create_dates(df_temp)
        except Exception as e:
            print(e)
        
        # 연령별로 컬럼 정리
        df_sum_per_day_집계구, df_sum_per_day_행정동 = combine_ages(df_temp)
        df_sum_per_day_집계구_list.append(df_sum_per_day_집계구)
        df_sum_per_day_행정동_list.append(df_sum_per_day_행정동)
    
    # 데이터 합치기
    df_concat_집계구 = pd.concat(df_sum_per_day_집계구_list)
    df_concat_행정동 = pd.concat(df_sum_per_day_행정동_list)

    return df_concat_집계구, df_concat_행정동

In [5]:
# 기준일ID 컬럼을 date형식으로 변환
# 년, 월 컬럼 생성
def create_dates(df_temp):
    df_temp.insert(0, '날짜', pd.to_datetime(df_temp['기준일ID'], format='%Y%m%d'))
    df_temp.insert(1, '년', df_temp['날짜'].dt.year)
    df_temp.insert(2, '월', df_temp['날짜'].dt.month)

    # 주말 여부 컬럼 생성
    df_temp.insert(3, "주말", df_temp['날짜'].dt.dayofweek > 4)

    # 기준일ID컬럼 drop
    df_temp.drop(columns=['기준일ID', '시간대구분'], inplace=True)
    # df_temp.drop(columns=['Unnamed: 33', 'Unnamed: 34', 'Unnamed: 35', 'Unnamed: 36','Unnamed: 37'], inplace=True)
    return df_temp

In [6]:
# 연령별로 컬럼 정리
def combine_ages(df_temp):
    cols = [
    '남자0세부터9세생활인구수', '여자0세부터9세생활인구수', 
    '남자10세부터14세생활인구수', '남자15세부터19세생활인구수', '여자10세부터14세생활인구수', '여자15세부터19세생활인구수', 
    '남자20세부터24세생활인구수', '남자25세부터29세생활인구수', '여자20세부터24세생활인구수', '여자25세부터29세생활인구수',
    '남자30세부터34세생활인구수', '남자35세부터39세생활인구수', '여자30세부터34세생활인구수', '여자35세부터39세생활인구수', 
    '남자40세부터44세생활인구수', '남자45세부터49세생활인구수', '여자40세부터44세생활인구수', '여자45세부터49세생활인구수', 
    '남자50세부터54세생활인구수', '남자55세부터59세생활인구수', '여자50세부터54세생활인구수', '여자55세부터59세생활인구수', 
    '남자60세부터64세생활인구수', '남자65세부터69세생활인구수', '여자60세부터64세생활인구수', '여자65세부터69세생활인구수', 
    '남자70세이상생활인구수', '여자70세이상생활인구수'
    ]
    df_temp['10세 이하'] = df_temp.남자0세부터9세생활인구수 + df_temp.여자0세부터9세생활인구수
    df_temp['10대'] = df_temp.남자10세부터14세생활인구수 + df_temp.남자15세부터19세생활인구수 + df_temp.여자10세부터14세생활인구수 + df_temp.여자15세부터19세생활인구수
    df_temp['20대'] = df_temp.남자20세부터24세생활인구수 + df_temp.남자25세부터29세생활인구수 + df_temp.여자20세부터24세생활인구수 + df_temp.여자25세부터29세생활인구수
    df_temp['30대'] = df_temp.남자30세부터34세생활인구수 + df_temp.남자35세부터39세생활인구수 + df_temp.여자30세부터34세생활인구수 + df_temp.여자35세부터39세생활인구수
    df_temp['40대'] = df_temp.남자40세부터44세생활인구수 + df_temp.남자45세부터49세생활인구수 + df_temp.여자40세부터44세생활인구수 + df_temp.여자45세부터49세생활인구수
    df_temp['50대'] = df_temp.남자50세부터54세생활인구수 + df_temp.남자55세부터59세생활인구수 + df_temp.여자50세부터54세생활인구수 + df_temp.여자55세부터59세생활인구수
    df_temp['60대'] = df_temp.남자60세부터64세생활인구수 + df_temp.남자65세부터69세생활인구수 + df_temp.여자60세부터64세생활인구수 + df_temp.여자65세부터69세생활인구수
    df_temp['70세 이상'] = df_temp.남자70세이상생활인구수 + df_temp.여자70세이상생활인구수
    df_temp.drop(columns=cols, inplace=True)
    df_sum_per_day_집계구 = df_temp.groupby(['날짜', '년', '월', '주말', '행정동코드', '집계구코드']).sum()
    df_sum_per_day_행정동 = df_temp.groupby(['날짜', '년', '월', '주말', '행정동코드']).sum()

    return df_sum_per_day_집계구, df_sum_per_day_행정동

In [7]:
#2019년
local_people_2019_df_집계구, local_people_2019_df_행정동 = data_process(local_people_2019_list, "./data/집계구 단위 서울 생활인구/local_people/2019/")

  0%|          | 0/351 [00:00<?, ?it/s]

In [ ]:
#2020년
local_people_2020_df_집계구, local_people_2020_df_행정동 = data_process(local_people_2020_list, "./data/집계구 단위 서울 생활인구/local_people/2020/")

In [ ]:
#2021년
local_people_2021_df_집계구, local_people_2021_df_행정동 = data_process(local_people_2021_list, "./data/집계구 단위 서울 생활인구/local_people/2021/")

In [17]:
local_people_2019_df_집계구.drop(columns=['Unnamed: 33', 'Unnamed: 34', 'Unnamed: 35', 'Unnamed: 36','Unnamed: 37'], inplace=True)


In [23]:
local_people_2019_df_행정동.drop(columns=['Unnamed: 33', 'Unnamed: 34', 'Unnamed: 35', 'Unnamed: 36','Unnamed: 37'], inplace=True)


In [24]:
local_people_2019_df_집계구.to_csv("./data/집계구 단위 서울 생활인구/2019일당일평균_집계구.csv")

In [25]:
local_people_2019_df_행정동.to_csv("./data/집계구 단위 서울 생활인구/2019일당일평균_행정동.csv")

In [37]:
local_people_2019_df_집계구

총생활인구수  10세 이하     10대  \
날짜         년    월  주말    행정동코드    집계구코드                                    
2019-01-01 2019 1  False 11110515 1101072010001   3891.0   236.0   220.0   
                                  1101072010002  10983.0   645.0   932.0   
                                  1101072010003   7545.0   438.0   608.0   
                                  1101072010004   3343.0   202.0   151.0   
                                  1101072010005   5302.0   342.0   390.0   
...                                                  ...     ...     ...   
2019-12-31 2019 12 False 11740700 1125071022201  16146.0  1214.0  1464.0   
                                  1125071022202   4217.0   306.0   210.0   
                                  1125071022501  19846.0  1189.0  1571.0   
                                  1125071022701   4536.0   419.0   324.0   
                                  1125071023701  11966.0   931.0  1104.0   

                                                    20대     30대     40대  \
날짜         년    월  주말    행정동코드    집계구코드                                   
2019-01-01 2019 1  False 11110515 1101072010001   465.0   435.0   663.0   
                                  1101072010002  1557.0  1638.0  1913.0   
                                  1101072010003  1081.0   900.0  1349.0   
                                  1101072010004   282.0   296.0   513.0   
                                  1101072010005   733.0   621.0   937.0   
...                                                 ...     ...     ...   
2019-12-31 2019 12 False 11740700 1125071022201  1820.0  2527.0  2662.0   
                                  1125071022202   353.0   609.0   652.0   
                                  1125071022501  1914.0  2641.0  2884.0   
                                  1125071022701   378.0   415.0   503.0   
                                  1125071023701  1366.0  1916.0  1994.0   

                                                    50대     60대  70세 이상  
날짜         년    월  주말    행정동코드    집계구코드                                  
2019-01-01 2019 1  False 11110515 1101072010001   508.0   229.0   447.0  
                                  1101072010002  1472.0   976.0  1532.0  
                                  1101072010003  1248.0   735.0   803.0  
                                  1101072010004   452.0   254.0   345.0  
                                  1101072010005   731.0   298.0   641.0  
...                                                 ...     ...     ...  
2019-12-31 2019 12 False 11740700 1125071022201  2360.0  1906.0  1866.0  
                                  1125071022202   524.0   372.0   416.0  
                                  1125071022501  2910.0  2998.0  3411.0  
                                  1125071022701   728.0   618.0   471.0  
                                  1125071023701  1734.0  1330.0  1258.0  

[6780158 rows x 9 columns]

In [43]:
pd.options.display.float_format = '{:,.0f}'.format

In [47]:
df_dong = local_people_2019_df_행정동.groupby(['년', '월', '주말', '행정동코드']).mean().reset_index()
df_dong.drop(columns='집계구코드', inplace=True)
df_dong

,년,월,주말,행정동코드,총생활인구수,10세 이하,10대,20대,30대,40대,50대,60대,70세 이상
0,2019,1,False,11110515,"393,389","22,095","31,747","55,760","62,022","73,075","54,134","35,318","46,148"
1,2019,1,False,11110530,"776,704","20,697","27,721","123,837","173,876","165,072","114,244","68,531","73,895"
2,2019,1,False,11110540,"157,556","3,441","10,246","24,154","29,987","28,814","24,688","16,805","17,634"
3,2019,1,False,11110550,"264,268","11,362","23,054","35,592","31,254","38,970","42,058","32,867","40,904"
4,2019,1,False,11110560,"430,351","26,350","38,179","42,789","42,539","58,514","70,096","58,896","74,300"
...,...,...,...,...,...,...,...,...,...,...,...,...,...
10171,2019,12,True,11740650,"675,499","44,050","41,766","92,167","112,312","101,932","101,465","83,327","76,850"
10172,2019,12,True,11740660,"659,218","38,433","55,628","84,222","100,720","106,994","100,689","79,970","70,731"
10173,2019,12,True,11740685,"1,288,847","82,883","93,527","159,933","189,504","195,212","203,194","173,226","148,016"
10174,2019,12,True,11740690,"194,670","11,626","17,732","15,055","20,827","26,976","24,678","19,163","25,157"


In [48]:
df_code = pd.read_excel("./data/집계구 단위 서울 생활인구/행정동코드_매핑정보_20200325.xlsx")
df_code.drop(0, inplace=True)
df_dong['행정동코드']=df_dong['행정동코드'].astype(str)
df_code['행자부행정동코드']=df_code['행자부행정동코드'].astype(str)
df_dong = pd.merge(df_dong, df_code, how='inner',left_on='행정동코드', right_on='행자부행정동코드')
df_dong

,년,월,주말,행정동코드,총생활인구수,10세 이하,10대,20대,30대,40대,50대,60대,70세 이상,통계청행정동코드,행자부행정동코드,시도명,시군구명,행정동명
0,2019,1,False,11110515,"393,389","22,095","31,747","55,760","62,022","73,075","54,134","35,318","46,148",1101072,11110515,서울,종로구,청운효자동
1,2019,1,True,11110515,"383,699","21,877","31,035","56,868","55,037","69,006","52,860","36,230","48,422",1101072,11110515,서울,종로구,청운효자동
2,2019,2,False,11110515,"370,118","20,885","29,996","52,583","58,694","68,774","50,634","32,472","42,593",1101072,11110515,서울,종로구,청운효자동
3,2019,2,True,11110515,"350,496","20,030","28,079","51,264","51,003","63,848","47,989","32,301","43,242",1101072,11110515,서울,종로구,청운효자동
4,2019,3,False,11110515,"409,506","27,178","51,132","55,846","61,799","72,298","51,629","33,935","42,776",1101072,11110515,서울,종로구,청운효자동
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10171,2019,10,True,11740700,"725,982","50,011","59,280","79,302","96,770","102,394","110,414","97,735","103,914",1125071,11740700,서울,강동구,둔촌2동
10172,2019,11,False,11740700,"746,113","50,377","60,167","77,219","98,473","105,720","111,483","100,611","115,891",1125071,11740700,서울,강동구,둔촌2동
10173,2019,11,True,11740700,"726,680","48,532","59,431","78,738","97,978","104,737","109,763","97,849","103,504",1125071,11740700,서울,강동구,둔촌2동
10174,2019,12,False,11740700,"741,778","49,136","59,633","78,151","97,075","105,063","109,431","100,563","116,146",1125071,11740700,서울,강동구,둔촌2동


In [68]:
df_dong.to_csv("./일당평균생활인구수_행정동.csv")

In [49]:
#총 일평균 생활인구
df_dong.pivot_table("총생활인구수", index=["시군구명","행정동명"],aggfunc = 'sum').sort_values(by=["총생활인구수"],ascending=False)

총생활인구수
시군구명 행정동명                  
강남구  역삼1동        58,736,706
영등포구 여의동         49,056,443
마포구  서교동         47,058,866
종로구  종로1.2.3.4가동 37,896,758
서초구  서초3동        36,906,542
...                     ...
종로구  교남동          3,919,072
     삼청동          3,886,807
영등포구 신길4동         3,834,838
양천구  신월6동         3,283,055
종로구  창신3동         2,345,835

[424 rows x 1 columns]

In [55]:
#순위 분석 : 각 평균에 따른 합 비교
age_lp=df_dong.filter(regex='\d|시군구명|행정동명')
age_lp

,10세 이하,10대,20대,30대,40대,50대,60대,70세 이상,시군구명,행정동명
0,"22,095","31,747","55,760","62,022","73,075","54,134","35,318","46,148",종로구,청운효자동
1,"21,877","31,035","56,868","55,037","69,006","52,860","36,230","48,422",종로구,청운효자동
2,"20,885","29,996","52,583","58,694","68,774","50,634","32,472","42,593",종로구,청운효자동
3,"20,030","28,079","51,264","51,003","63,848","47,989","32,301","43,242",종로구,청운효자동
4,"27,178","51,132","55,846","61,799","72,298","51,629","33,935","42,776",종로구,청운효자동
...,...,...,...,...,...,...,...,...,...,...
10171,"50,011","59,280","79,302","96,770","102,394","110,414","97,735","103,914",강동구,둔촌2동
10172,"50,377","60,167","77,219","98,473","105,720","111,483","100,611","115,891",강동구,둔촌2동
10173,"48,532","59,431","78,738","97,978","104,737","109,763","97,849","103,504",강동구,둔촌2동
10174,"49,136","59,633","78,151","97,075","105,063","109,431","100,563","116,146",강동구,둔촌2동


In [57]:
a=age_lp.groupby(["시군구명","행정동명"]).mean()
a["1위"]=a.idxmax(axis=1)
a

10세 이하    10대     20대     30대     40대     50대    60대  70세 이상   1위
시군구명 행정동명                                                                   
강남구  개포1동  11,925 13,919  11,670  20,556  28,787  27,015 24,990  24,892  40대
     개포2동  25,962 56,116  37,873  43,714  75,453  67,640 58,940  51,543  40대
     개포4동  22,523 36,195  41,729  60,099  65,865  59,140 48,450  37,374  40대
     논현1동  22,907 40,140 249,211 270,219 172,789 104,921 81,530  73,521  30대
     논현2동  38,689 43,868 199,472 254,801 185,008 111,319 80,773  77,989  30대
...           ...    ...     ...     ...     ...     ...    ...     ...  ...
중랑구  상봉2동  29,167 31,797 109,549 110,984  91,719 101,577 77,645  75,042  30대
     신내1동  84,608 66,680  86,149 105,983 116,990 125,771 90,757  92,877  50대
     신내2동  31,875 38,353  46,895  57,178  62,364  68,458 52,447  63,365  50대
     중화1동  37,833 31,631  46,617  57,442  51,096  59,797 46,554  52,170  50대
     중화2동  37,717 37,244  82,697  87,697  90,204 117,504 88,827  92,868  50대

[424 rows x 9 columns]

In [59]:
a.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 424 entries, ('강남구', '개포1동') to ('중랑구', '중화2동')
Data columns (total 9 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   10세 이하  424 non-null    float64
 1   10대     424 non-null    float64
 2   20대     424 non-null    float64
 3   30대     424 non-null    float64
 4   40대     424 non-null    float64
 5   50대     424 non-null    float64
 6   60대     424 non-null    float64
 7   70세 이상  424 non-null    float64
 8   1위      424 non-null    object 
dtypes: float64(8), object(1)
memory usage: 34.7+ KB


## 행정동 별 나이대 순위: 모든 순위

In [62]:
a=age_lp.groupby(["시군구명","행정동명"]).mean()
a_list=a.apply(np.argsort,axis=1)
a_list.values

array([[2, 0, 1, ..., 6, 5, 4],
       [0, 2, 3, ..., 6, 5, 4],
       [0, 1, 7, ..., 5, 3, 4],
       ...,
       [0, 1, 2, ..., 4, 7, 5],
       [1, 0, 6, ..., 7, 3, 5],
       [1, 0, 2, ..., 4, 7, 5]])

In [63]:
a_cols=a.columns.to_numpy()[a_list.values[:,::-1]]
a_cols

array([['40대', '50대', '60대', ..., '10대', '10세 이하', '20대'],
       ['40대', '50대', '60대', ..., '30대', '20대', '10세 이하'],
       ['40대', '30대', '50대', ..., '70세 이상', '10대', '10세 이하'],
       ...,
       ['50대', '70세 이상', '40대', ..., '20대', '10대', '10세 이하'],
       ['50대', '30대', '70세 이상', ..., '60대', '10세 이하', '10대'],
       ['50대', '70세 이상', '40대', ..., '20대', '10세 이하', '10대']],
      dtype=object)

In [66]:
len(a_cols[0])

8

In [67]:
a[['1위','2위','3위','4위','5위','6위','7위','8위']]=pd.DataFrame(a_cols, index=a.index)
a

10세 이하    10대     20대     30대     40대     50대    60대  70세 이상   1위  \
시군구명 행정동명                                                                      
강남구  개포1동  11,925 13,919  11,670  20,556  28,787  27,015 24,990  24,892  40대   
     개포2동  25,962 56,116  37,873  43,714  75,453  67,640 58,940  51,543  40대   
     개포4동  22,523 36,195  41,729  60,099  65,865  59,140 48,450  37,374  40대   
     논현1동  22,907 40,140 249,211 270,219 172,789 104,921 81,530  73,521  30대   
     논현2동  38,689 43,868 199,472 254,801 185,008 111,319 80,773  77,989  30대   
...           ...    ...     ...     ...     ...     ...    ...     ...  ...   
중랑구  상봉2동  29,167 31,797 109,549 110,984  91,719 101,577 77,645  75,042  30대   
     신내1동  84,608 66,680  86,149 105,983 116,990 125,771 90,757  92,877  50대   
     신내2동  31,875 38,353  46,895  57,178  62,364  68,458 52,447  63,365  50대   
     중화1동  37,833 31,631  46,617  57,442  51,096  59,797 46,554  52,170  50대   
     중화2동  37,717 37,244  82,697  87,697  90,204 117,504 88,827  92,868  50대   

               2위      3위      4위      5위      6위      7위      8위  
시군구명 행정동명                                                          
강남구  개포1동     50대     60대  70세 이상     30대     10대  10세 이하     20대  
     개포2동     50대     60대     10대  70세 이상     30대     20대  10세 이하  
     개포4동     30대     50대     60대     20대  70세 이상     10대  10세 이하  
     논현1동     20대     40대     50대     60대  70세 이상     10대  10세 이하  
     논현2동     20대     40대     50대     60대  70세 이상     10대  10세 이하  
...           ...     ...     ...     ...     ...     ...     ...  
중랑구  상봉2동     20대     50대     40대     60대  70세 이상     10대  10세 이하  
     신내1동     40대     30대  70세 이상     60대     20대  10세 이하     10대  
     신내2동  70세 이상     40대     30대     60대     20대     10대  10세 이하  
     중화1동     30대  70세 이상     40대     20대     60대  10세 이하     10대  
     중화2동  70세 이상     40대     60대     30대     20대  10세 이하     10대  

[424 rows x 16 columns]

In [ ]:
def name_dong(df):
    df_code = pd.read_excel("./data/집계구 단위 서울 생활인구/행정동코드_매핑정보_20200325.xlsx")
    df_code.drop(0, inplace=True)
    df['행정동코드']=df['행정동코드'].astype(str)
    df_code['행자부행정동코드']=df_code['행자부행정동코드'].astype(str)
    df = pd.merge(df, df_code, how='inner',left_on='행정동코드', right_on='행자부행정동코드')
    return df
